**General description**

This notebook has been developed by Ana Teresa Lopez Jimenez @ LSHTM

It has been used in the preprint: High-content superresolution microscopy and deep learning assisted analysis reveals host and bacterial heterogeneity during Shigella infection. Ana T. López-Jiménez, Dominik Brokatzky, Kamla Pillay, Tyrese Williams, Gizem Özbaykal Güler and Serge Mostowy (2024)


This notebook allows to classify single vs. clumped rod-shape bacteria.

**Preparation, import of required packages**

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)

2.15.0


In [ ]:
!pip install pyyaml h5py

**Data loading from google drive**


In [ ]:
from google.colab import drive
root = '/content/gdrive/'
drive.mount( root )

Mounted at /content/gdrive/


In [ ]:
singles_dir_path = r'/My Drive/folder' # write here directory where files to be analysed are
os.makedirs(root+singles_dir_path, exist_ok=True)
os.listdir(root+singles_dir_path)


['all_tiff_2_crop_allchannels_singles']

**Importing the Model**

For this step, it is required to add the trained model "model_classification_single_vs_clump.hdf5" to Google drive, and specify the correct folder here below to retrieve it

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization

In [ ]:

new_model = tf.keras.models.load_model('/content/gdrive/My Drive/folder/model_classification_single_vs_clump.hdf5') # Specify the folder containing the model
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 4)       40        
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 4)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 8)         296       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 16)        1168      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 16)        0

In [ ]:
new_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 32
IMG_SHAPE  = 128 # Our training data consists of images with width of 128 pixels and height of 128 pixels.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255)

image_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=root+singles_dir_path,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 color_mode="grayscale",
                                                 class_mode='binary',
                                                 shuffle=False)

Found 626 images belonging to 1 classes.


**Applying the model and saving results**

In this step the model will be applied to generate predictions (0, 1).
The results can be saved as a .txt document in the folder specificied here below.

In [ ]:
new_data = []
file_names = image_data_gen.filenames
predict_x=new_model.predict(image_data_gen)
classes_x=[1 * (x[0]>=0.5) for x in predict_x]

new_data = np.array([file_names, classes_x]).T
print(new_data)
np.savetxt('/content/gdrive/My Drive/folder/Results_singles.txt', new_data, fmt='%s') # Specify here the output folder


20/20 [==============================] - 4s 122ms/step
[['all_tiff_2_crop_allchannels_singles/231011_p0042-01-Scene-01-P1-E02-AiPr-301n0.tif'
  '0']
 ['all_tiff_2_crop_allchannels_singles/231011_p0042-01-Scene-01-P1-E02-AiPr-301n1.tif'
  '1']
 ['all_tiff_2_crop_allchannels_singles/231011_p0042-01-Scene-01-P1-E02-AiPr-301n2.tif'
  '0']
 ...
 ['all_tiff_2_crop_allchannels_singles/231011_p0042-02-Scene-220-P10-G11-AiPr-597n7.tif'
  '1']
 ['all_tiff_2_crop_allchannels_singles/231011_p0042-02-Scene-220-P10-G11-AiPr-597n8.tif'
  '1']
 ['all_tiff_2_crop_allchannels_singles/231011_p0042-02-Scene-220-P10-G11-AiPr-597n9.tif'
  '0']]
